## Import

In [58]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

In [59]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [60]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [61]:
torch.cuda.is_available()

True

In [62]:
torch.__version__

'2.0.1+cu118'

In [63]:
# torch.cuda.empty_cache()

## Hyperparameter Setting

In [64]:
CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습  초기는 90일이였음 
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':110,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4096,
    'SEED':41
}

In [65]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

In [66]:
train_data = pd.read_csv('E:/LG/LG_data/train.csv').drop(columns=['ID', '제품'])

In [67]:
brand_keyword_cnt = pd.read_csv('E:/LG/LG_data/brand_keyword_cnt.csv')

In [68]:
train_data.head(10)

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,7,21,16,...,0,0,0,0,0,0,0,0,0,0
7,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,15,16,23,...,0,0,0,0,0,0,0,0,0,1
9,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15890 entries, 0 to 15889
Columns: 463 entries, 대분류 to 2023-04-04
dtypes: int64(459), object(4)
memory usage: 56.1+ MB


In [70]:
train_data.describe()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
count,15890.000000,15890.000000,15890.00000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,...,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000
mean,12.887476,10.418880,9.01309,9.190938,11.204216,12.486281,12.933103,12.832599,13.326935,12.049843,...,0.501699,0.483260,0.473694,9.498175,9.370044,7.706293,7.123033,6.975456,9.040340,10.031026
std,183.612376,149.663362,95.82452,86.274138,92.072773,108.478567,135.615709,233.900622,242.761978,242.326264,...,8.853648,8.911203,8.567765,93.854552,90.632349,68.413621,68.629350,59.826757,73.637191,103.256072
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,15056.000000,14320.000000,6064.00000,4470.000000,6370.000000,8210.000000,9712.000000,24512.000000,24032.000000,24960.000000,...,700.000000,660.000000,660.000000,4308.000000,4596.000000,3960.000000,4224.000000,2700.000000,3045.000000,6048.000000


In [71]:
brand_keyword_cnt.head(2)

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.84131,0.91383,1.45053,2.42239,1.87119,1.58108,1.23295,1.17493,1.14592,...,0.31911,0.39164,0.37713,0.49318,0.07252,0.29010,0.31911,0.23208,0.33362,0.44966
1,B002-00002,12.64868,20.27850,15.33217,12.75021,13.56251,13.70757,11.93791,15.56425,14.08471,...,10.26979,11.96692,10.64693,10.41485,10.48738,9.48651,9.28343,10.42935,11.15462,11.38671


### 데이터 전처리

In [72]:
# # 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
# numeric_cols = train_data.columns[4:]
# # 칵 column의 min 및 max 계산
# min_values = train_data[numeric_cols].min(axis=1)
# max_values = train_data[numeric_cols].max(axis=1)
# # 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
# ranges = max_values - min_values
# ranges[ranges == 0] = 1
# # min-max scaling 수행
# train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# # max와 min 값을 dictionary 형태로 저장
# scale_min_dict = min_values.to_dict()
# scale_max_dict = max_values.to_dict()

# 새로 시도해볼 전처리 방식 - 분모를 0이 되는 것을 방지

## Box-Cox 변환

In [73]:
# import pandas as pd
# from scipy.stats import boxcox

# numeric_cols = train_data.columns[4:]

# # Box-Cox 변환 수행
# transformed_data = pd.DataFrame()
# for col_name in numeric_cols:
#     col_data = train_data[col_name]
#     if col_data.min() > 0:  # Only apply Box-Cox to positive values
#         transformed_col, _ = boxcox(col_data)
#     else:
#         transformed_col = col_data
#     transformed_data[col_name] = transformed_col

# # 변환된 데이터로 업데이트
# train_data[numeric_cols] = transformed_data

# print(train_data.head())


In [74]:
# import pandas as pd
# from scipy.stats import boxcox
# import numpy as np

# # Train 데이터에 대해 Box-Cox 변환을 수행하여 train_data 업데이트
# numeric_cols = train_data.columns[4:]
# scale_lambda_dict = {}  # 람다 값을 저장할 딕셔너리 초기화

# transformed_data = pd.DataFrame()
# for col_name in numeric_cols:
#     col_data = train_data[col_name]
#     if col_data.min() > 0:
#         transformed_col, lambda_value = boxcox(col_data)
#         scale_lambda_dict[col_name] = lambda_value  # 람다 값을 저장
#     else:
#         transformed_col = col_data
#     transformed_data[col_name] = transformed_col

# train_data[numeric_cols] = transformed_data

# print(train_data.head())


## 로그 변환

In [75]:
# import pandas as pd
# import numpy as np

# numeric_cols = train_data.columns[4:]

# # 로그 변환 수행
# log_transformed_data = np.log1p(train_data[numeric_cols])

# # 로그 변환된 데이터로 업데이트
# train_data[numeric_cols] = log_transformed_data

# print(train_data.head())


# Z-score 전처리 방법

In [76]:
# import pandas as pd

# # 더 작은 값을 추가하여 분모가 0인 경우 방지하는 상수 정의
# EPSILON = 1e-8

# numeric_cols = train_data.columns[4:]

# # 각 행의 평균과 표준편차 계산
# mean_values = train_data[numeric_cols].mean(axis=1)
# std_values = train_data[numeric_cols].std(axis=1)

# # 표준편차가 0인 경우 EPSILON 더해주기
# std_values = std_values.apply(lambda x: x if x > 0 else EPSILON)

# # Z-score 표준화 수행
# z_score_data = train_data[numeric_cols].subtract(mean_values, axis=0).div(std_values, axis=0)

# # 표준화된 데이터로 업데이트
# train_data[numeric_cols] = z_score_data

# # 평균과 표준편차 값을 dictionary 형태로 저장
# scale_mean_dict = mean_values.to_dict()
# scale_std_dict = std_values.to_dict()

# print(train_data.head())


## min -max

In [77]:
import pandas as pd

# 더 작은 값을 추가하여 분모가 0인 경우 방지하는 상수 정의
EPSILON = 1e-8

numeric_cols = train_data.columns[4:]

# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 EPSILON 더해주기
ranges = max_values - min_values + EPSILON

# min-max scaling 수행
scaled_data = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

# 스케일링된 데이터로 업데이트
train_data[numeric_cols] = scaled_data

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()


In [78]:
train_data.tail()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
15885,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15886,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02439,0.0,0.01626,0.03252,0.00813,0.00813,0.024390
15887,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15888,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.142857
15889,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000


In [79]:
# # Data Scaling
# scale_max_dict = {}
# scale_min_dict = {}

# for idx in tqdm(range(len(train_data))):
#     maxi = np.max(train_data.iloc[idx,4:])
#     mini = np.min(train_data.iloc[idx,4:])
    
#     if maxi == mini :
#         train_data.iloc[idx,4:] = 0
#     else:
#         train_data.iloc[idx,4:] = (train_data.iloc[idx,4:] - mini) / (maxi - mini)
    
#     scale_max_dict[idx] = maxi
#     scale_min_dict[idx] = mini

In [80]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [81]:
# class CustomDataset(Dataset):
#     def __init__(self, data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'], is_inference=False):
#         self.data = data.values # convert DataFrame to numpy array
#         self.train_size = train_size
#         self.predict_size = predict_size
#         self.window_size = self.train_size + self.predict_size
#         self.is_inference = is_inference

#     def __len__(self):
#         if self.is_inference:
#             return len(self.data)
#         else:
#             return self.data.shape[0] * (self.data.shape[1] - self.window_size - 3)

#     def __getitem__(self, idx):
#         if self.is_inference:
#             # 추론 시
#             encode_info = self.data[idx, :4]
#             window = self.data[idx, -self.train_size:]
#             input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window))
#             return input_data
#         else:
#             # 학습 시
#             row = idx // (self.data.shape[1] - self.window_size - 3)
#             col = idx % (self.data.shape[1] - self.window_size - 3)
#             encode_info = self.data[row, :4]
#             sales_data = self.data[row, 4:]
#             window = sales_data[col : col + self.window_size]
#             input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window[:self.train_size]))
#             target_data = window[self.train_size:]
#             return input_data, target_data

In [82]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [83]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [84]:
train_data.tail()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
15885,2,7,41,3169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15886,2,7,43,3169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02439,0.0,0.01626,0.03252,0.00813,0.00813,0.024390
15887,2,7,43,3169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15888,2,7,43,3169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.142857
15889,1,3,19,3169,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000


In [85]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

100%|██████████| 15890/15890 [00:18<00:00, 866.45it/s]


In [86]:
data_len = len(train_input)
val_ratio = 0.1
test_ratio = 0.1

val_len = int(data_len * val_ratio)
test_len = int(data_len * test_ratio)

val_input = train_input[-val_len:]
val_target = train_target[-val_len:]

train_input = train_input[:-val_len - test_len]
train_target = train_target[:-val_len - test_len]


In [87]:
# # Train / Validation Split
# data_len = len(train_input)
# val_input = train_input[-int(data_len*0.2):]
# val_target = train_target[-int(data_len*0.2):]
# train_input = train_input[:-int(data_len*0.1)]
# train_target = train_target[:-int(data_len*0.1)]

In [88]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4105976, 120, 5),
 (4105976, 21),
 (513247, 120, 5),
 (513247, 21),
 (15890, 120, 5))

### Custom Dataset

In [89]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [90]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [91]:
# from torch.utils.data import Dataset, DataLoader, random_split
# # CustomDataset 인스턴스 생성
# dataset = CustomDataset(train_data)

# # 전체 데이터셋의 크기
# total_size = len(dataset)

# # 분리할 데이터셋의 크기 계산
# train_size = int(total_size * 0.9)
# val_size = total_size - train_size

# # random_split 함수를 사용해 데이터셋 분리
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# # DataLoader 인스턴스 생성
# train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

### 모델 선언

## GRU 이용

In [92]:
# import torch.nn as nn
# import torch

# class BaseModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE']):
#         super(BaseModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Dropout(),
#             nn.Linear(hidden_size // 2, output_size)
#         )

#         self.actv = nn.ReLU()

#     def forward(self, x):
#         # x shape: (B, TRAIN_WINDOW_SIZE, 5)
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)

#         # GRU layer
#         gru_out, hidden = self.gru(x, hidden)

#         # Only use the last output sequence
#         last_output = gru_out[:, -1, :]

#         # Fully connected layer
#         output = self.actv(self.fc(last_output))

#         return output.squeeze(1)

#     def init_hidden(self, batch_size, device):
#         # Initialize hidden state for all GRU layers
#         return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)


## 업그레이드 GRU

In [93]:
# import torch.nn as nn
# import torch

# class ImprovedModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE']):
#         super(ImprovedModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
#         self.dropout = nn.Dropout(0.2)  # Adding dropout after GRU
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Linear(hidden_size // 2, output_size)
#         )
#         self.actv = nn.ReLU()  # Using LeakyReLU activation

#     def forward(self, x):
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)

#         gru_out, hidden = self.gru(x, hidden)
#         gru_out = self.dropout(gru_out)  # Applying dropout

#         last_output = gru_out[:, -1, :]

#         output = self.actv(self.fc(last_output))

#         return output.squeeze(1)

#     def init_hidden(self, batch_size, device):
#         return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

In [94]:
# import torch.nn as nn
# import torch

# class ImprovedModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE'], dropout_prob=0.2):
#         super(ImprovedModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
#         self.dropout = nn.Dropout(dropout_prob)  # Adding dropout after GRU
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Linear(hidden_size // 2, output_size)
#         )
#         self.actv = nn.ReLU()  # Using LeakyReLU activation
        

#     def forward(self, x):
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)

#         gru_out, hidden = self.gru(x, hidden)
#         gru_out = self.dropout(gru_out)  # Applying dropout

#         last_output = gru_out[:, -1, :]

#         output = self.actv(self.fc(last_output))

#         return output.squeeze(1)

#     def init_hidden(self, batch_size, device):
#         return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)


# ATTENDTION적용 GRU

In [95]:
# class ImprovedModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE'], dropout_prob=0.2):
#         super(ImprovedModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
#         self.attention = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size),  # Adjust as needed
#             nn.Tanh(),
#             nn.Linear(hidden_size, 1)
#         )
#         self.dropout = nn.Dropout(dropout_prob)
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Linear(hidden_size // 2, output_size)
#         )
#         self.actv = nn.ReLU()

#     def forward(self, x):
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)

#         gru_out, hidden = self.gru(x, hidden)
#         gru_out = self.dropout(gru_out)

#         attention_weights = self.attention(gru_out).squeeze(-1)
#         attention_weights = torch.softmax(attention_weights, dim=1)
#         context = torch.bmm(attention_weights.unsqueeze(1), gru_out).squeeze(1)

#         output = self.actv(self.fc(context))

#         return output.squeeze(1)

#     def init_hidden(self, batch_size, device):
#         return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)


# Layer Normalization

In [96]:
class ImprovedModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE'], dropout_prob=0.1):
        super(ImprovedModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.ln = nn.LayerNorm(hidden_size)  # Layer Normalization
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        gru_out, hidden = self.gru(x, hidden)
        gru_out = self.dropout(gru_out)
        gru_out = self.ln(gru_out)  # Applying Layer Normalization

        last_output = gru_out[:, -1, :]

        output = self.actv(self.fc(last_output))

        return output.squeeze(1)
    def init_hidden(self, batch_size, device):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)



### 모델 학습

In [97]:
# def train(model, optimizer, train_loader, val_loader, device, scheduler = None):
#     model.to(device)
#     criterion = nn.MSELoss().to(device)
#     best_loss = 9999999
#     best_model = None
    
    
    
#     for epoch in range(1, CFG['EPOCHS']+1):
#         model.train()
#         train_loss = []
#         train_mae = []
        
#         for X, Y in tqdm(iter(train_loader)):
#             X = X.to(device)
#             Y = Y.to(device)

            
#             optimizer.zero_grad()
            
#             output = model(X)
#             loss = criterion(output, Y)
            
#             loss.backward()
#             optimizer.step()
            
#             train_loss.append(loss.item())

#         if scheduler is not None:
#             scheduler.step()  # Update learning rate
        
#         val_loss = validation(model, val_loader, criterion, device)
#         print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        


#         # # 학습 루프 안에서
#         # if best_loss > val_loss:
#         #     best_loss = val_loss
#         #     best_model = model
#         #     print('Model Saved')

#         # 학습이 끝난 후
#     return model  # 모든 모델을 반환




In [98]:
# def train(model, optimizer, train_loader, val_loader, device, scheduler=None):
#     model.to(device)
#     criterion = nn.MSELoss().to(device)
#     best_loss = float('inf')
#     best_model = None
#     early_stopping_counter = 0
#     max_early_stopping = 30  # Max number of consecutive epochs with increasing validation loss
    
#     for epoch in range(1, CFG['EPOCHS'] + 1):
#         model.train()
#         train_loss = []
#         train_mae = []

#         for X, Y in tqdm(iter(train_loader)):
#             X = X.to(device)
#             Y = Y.to(device)

#             optimizer.zero_grad()

#             output = model(X)
#             loss = criterion(output, Y)

#             loss.backward()
#             optimizer.step()

#             train_loss.append(loss.item())


        
#         val_loss = validation(model, val_loader, criterion, device)
#         print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

#         if scheduler is not None:
#             scheduler.step(val_loss)  # Update learning rate based on scheduler's policy
        
#         if val_loss < best_loss:
#             best_loss = val_loss
#             best_model = model
#             early_stopping_counter = 0
#         else:
#             early_stopping_counter += 1
#             if early_stopping_counter >= max_early_stopping:
#                 print(f'Early stopping triggered at epoch {epoch}')
#                 break

#     return best_model


In [99]:
def train(model, optimizer, train_loader, val_loader, device, scheduler=None):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = float('inf')
    best_model = None
    early_stopping_counter = 0
    max_early_stopping = 30  # Max number of consecutive epochs with increasing validation loss # 이번 코드는 17
    
    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        train_mae = []

        for X, Y in tqdm(iter(train_loader)):
            # X = X.to(device)
            # Y = Y.to(device)
            X = X.float().to(device)
            Y = Y.float().to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())


        
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(val_loss)  # Update learning rate based on scheduler's policy
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= max_early_stopping:
                print(f'Early stopping triggered at epoch {epoch}')
                break

    return best_model


In [100]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            # X = X.to(device)
            # Y = Y.to(device)
            X = X.float().to(device)
            Y = Y.float().to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

In [101]:
# def validation(model, val_loader, criterion, device):
#     model.eval()
#     val_loss = []
    
#     with torch.no_grad():
#         for X, Y in tqdm(iter(val_loader)):
#             X = X.to(device)
#             Y = Y.to(device)
            
#             output = model(X)
#             loss = criterion(output, Y)
            
#             val_loss.append(loss.item())
#     return np.mean(val_loss)

## Run !!

# 0.7//    2   지금은 0.7 // 1 ㅣㅁ

# max_early_stopping = 10  # Max number of consecutive epochs with increasing validation loss 다음에는 0.7 0.9 비교하고 이거 늘리자 한 20 ~ 30

In [102]:
import torch.optim.lr_scheduler as lr_scheduler

model = ImprovedModel() # BaseModel() 
optimizer = torch.optim.RAdam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, factor=0.9, verbose=True)  # 지금 돌린 코드는 patience = 2 이었음
infer_model = train(model, optimizer, train_loader, val_loader, device, scheduler)


  0%|          | 0/1003 [00:00<?, ?it/s]

100%|██████████| 126/126 [00:27<00:00,  4.60it/s]


Epoch : [1] Train Loss : [0.03213] Val Loss : [0.02847]


100%|██████████| 126/126 [00:27<00:00,  4.65it/s]


Epoch : [2] Train Loss : [0.03034] Val Loss : [0.02964]


...

Epoch : [46] Train Loss : [0.01437] Val Loss : [0.01750]
Epoch 00046: reducing learning rate of group 0 to 1.6677e-05.


100%|██████████| 126/126 [00:26<00:00,  4.75it/s]


Epoch : [47] Train Loss : [0.01432] Val Loss : [0.01744]


100%|██████████| 126/126 [00:26<00:00,  4.73it/s]

Epoch : [48] Train Loss : [0.01430] Val Loss : [0.01762]
Epoch 00048: reducing learning rate of group 0 to 1.5009e-05.
Early stopping triggered at epoch 48


## 모델 추론

In [103]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [104]:
# test_dataset = CustomDataset(data= train_data, is_inference=True)
# test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [105]:
# def inference(model, test_loader, device):
#     predictions = []
    
#     with torch.no_grad():
#         for X in tqdm(iter(test_loader)):
#             X = X.to(device)
            
#             output = model(X)
            
#             # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
#             output = output.cpu().numpy()
            
#             predictions.extend(output)
    
#     return np.array(predictions)

In [106]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            # X = X.to(device)
            X = X.float().to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [107]:
pred = inference(infer_model, test_loader, device)

100%|██████████| 4/4 [00:00<00:00,  6.22it/s]


## min max 역 스케일링

In [108]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

## z-score 역 스케일링

In [109]:
# # 추론 결과 역 스케일링
# for idx in range(len(pred)):
#         pred[idx, :] = pred[idx, :] * scale_std_dict[col_name] + scale_mean_dict[col_name]

# # 결과 후처리
# pred = np.round(pred, 0).astype(int)

# box-cox

In [110]:
# # 이후 예측 결과의 역 스케일링
# for idx in range(len(pred)):
#     for col_idx, col_name in enumerate(numeric_cols):
#         if col_name in scale_lambda_dict:  # 변환한 열에 대해서만 역 변환 수행
#             pred[idx, col_idx] = np.expm1(pred[idx, col_idx])  # 역 로그 변환
#             pred[idx, col_idx] = (pred[idx, col_idx] ** (1.0 / scale_lambda_dict[col_name])) - 1  # 역 Box-Cox 변환

# # 결과 후처리
# pred = np.round(pred, 0).astype(int)

In [111]:
pred.shape

(15890, 21)

## Submission

In [112]:
submit = pd.read_csv('E:/LG/LG_data/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,1,0,0,0,...,1,1,1,0,0,0,1,1,1,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
submit.to_csv('E:/LG/LG_data/models/baseline_submit_epoch_150_RAdam_GRUNOnozition_2_LAYER_2_0.3.csv', index=False)